In [2]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [6]:
results = requests.get("https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2017-01-29&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [8]:
results.status_code

200

In [18]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "mass-market-paperback"   # you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = "keys.json"

bestsellers = []
weeks = []
start_date = datetime.date(2017, 1, 29)  # Starting date
num_weeks = 260  # Number of weeks to fetch

for i in range(num_weeks):
    curr_date = start_date - datetime.timedelta(weeks=i)
    print(curr_date)
    url = "%s?list=%s&published-date=%s&api-key=%s" % (BOOKS_ROOT, LIST, curr_date, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    
    if r.status_code == 200:
        results = r.json()
        for result in results['results']:
            for book_details in result['book_details']:
                book_info = {
                    'title': book_details['title'],
                    'description': book_details['description'],
                    'contributor': book_details['contributor'],
                    'author': book_details['author'],
                    'contributor_note': book_details['contributor_note'],
                    'price': book_details['price'],
                    'age_group': book_details['age_group'],
                    'publisher': book_details['publisher'],
                    'primary_isbn13': book_details['primary_isbn13'],
                    'primary_isbn10': book_details['primary_isbn10'],
                    'rank': result['rank'],
                    'rank_last_week': result['rank_last_week'],
                    'weeks_on_list': result['weeks_on_list'],
                    'asterisk': result['asterisk'],
                    'dagger': result['dagger']
                }
                bestsellers.append(book_info)
                weeks.append(str(curr_date))
    else:
        print("API call did not work!")
        print(r)
    
    time.sleep(15)

2017-01-29
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2017-01-29&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-22
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2017-01-22&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-15
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2017-01-15&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-08
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2017-01-08&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2017-01-01
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2017-01-01&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2016-12-25
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperback&published-date=2016-12-25&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2016-12-18
https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-paperb

In [19]:
mass_market_paperback_weekly = pd.json_normalize(bestsellers)

In [20]:
mass_market_paperback_weekly.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,SWEET TOMORROWS,The journeys of the characters at the Rose Har...,by Debbie Macomber,Debbie Macomber,,0.00,,Ballantine,9780553391862,0553391860,1,1,3,0,0
1,THE MURDER HOUSE,When bodies are found at a Hamptons estate whe...,by James Patterson and David Ellis,James Patterson and David Ellis,,0.00,,Vision,9781455589913,1455589918,2,2,4,0,0
2,APPRENTICE IN DEATH,Lt. Eve Dallas of the N.Y.P.D. is on the case ...,by J. D. Robb,J D Robb,,0.00,,Berkley,9781101987995,1101987995,3,3,2,0,0
3,A DOG'S PURPOSE,"From stray mutt to golden-haired puppy, a dog ...",by W. Bruce Cameron,W Bruce Cameron,,0.00,,Forge,9780765388100,0765388103,4,5,6,0,0
4,MULBERRY MOON,Ben Sterling returns home to Mystic Creek to s...,by Catherine Anderson,Catherine Anderson,,0.00,,Jove,9780451488022,0451488024,5,4,2,0,0


In [21]:
mass_market_paperback_weekly.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
4820,ON LAVENDER LANE,"Settling into civilian life in Shelter Bay, a ...",by JoAnn Ross,JoAnn Ross,,7.99,,Signet,9780451235435,0451235436,16,0,4,0,0
4821,A CLASH OF KINGS,From the citadel of Dragonstone to the shores ...,by George R.R. Martin,George RR Martin,,8.99,,Bantam,9780553579901,0553579908,17,0,36,0,0
4822,MINDING FRANKIE,With the help of a devoted network of friends ...,by Maeve Binchy,Maeve Binchy,,7.99,,Anchor,9780307475497,0307475492,18,0,5,0,0
4823,HIDDEN SUMMIT,"Sick of her ex-husband and his new wife, Lesli...",by Robyn Carr,Ron Carr,,7.99,,Mira,9780778313007,077831300X,19,0,5,0,0
4824,MOONLIGHT IN THE MORNING,A woman leaves the ruthless New York art world...,by Jude Deveraux,Jude Deveraux,,7.99,,Pocket Star,9781416509745,1416509747,20,0,5,1,0


In [22]:
len(mass_market_paperback_weekly)

4825

In [23]:
len(weeks)

4825

In [24]:
#add a week column
mass_market_paperback_weekly['week'] = weeks

#and check to make sure it worked correctly
mass_market_paperback_weekly.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
4805,BONNIE,The forensic sculptor Eve Duncan learns more a...,by Iris Johansen,Iris Johansen,,7.99,,St. Martin’s,9780312651282,0312651287,1,0,1,0,0,2012-02-12
4806,THE GIRL WHO PLAYED WITH FIRE,In the second volume of the Millennium trilogy...,by Stieg Larsson,Stieg Larsson,,7.99,,Vintage Crime/Black Lizard,9780307476159,0307476154,2,0,71,0,0,2012-02-12
4807,THE GIRL WITH THE DRAGON TATTOO,A hacker and a journalist investigate the disa...,by Stieg Larsson,Stieg Larsson,,7.99,,Vintage Crime/Black Lizard,9780307949486,0307949486,3,0,83,0,0,2012-02-12
4808,A LADY NEVER SURRENDERS,When the youngest Sharpe sister hatches a plan...,by Sabrina Jeffries,Sabrina Jeffries,,7.99,,Pocket,9781451642452,1451642458,4,0,1,0,0,2012-02-12
4809,ONE FOR THE MONEY,The Jersey girl Stephanie Plum enters the busi...,by Janet Evanovich,Janet Evanovich,,8.99,,St. Martin’s,9780312547004,0312547005,5,0,6,0,0,2012-02-12
4810,THE JEFFERSON KEY,The ex-government operative Cotton Malone foil...,by Steve Berry,Steve Berry,,9.99,,Ballantine,9780345505521,0345505522,6,0,5,0,0,2012-02-12
4811,TOM CLANCY PRESENTS ACT OF VALOR,When the rescue of a C.I.A. operative leads to...,novelization by Dick Couch and George Galdorisi,novelization Dick Couch and George Galdorisi,,9.99,,Berkley,9780425259351,0425259358,7,0,2,0,0,2012-02-12
4812,SMOKIN’ SEVENTEEN,No one is sure why dead bodies are turning up ...,by Janet Evanovich,Janet Evanovich,,8.99,,Bantam,9780345527707,0345527704,8,0,11,0,0,2012-02-12
4813,YOU ... AGAIN,"A reissue of two novels, “Baby Blessed"" and “Y...",by Debbie Macomber,Debbie Macomber,,7.99,,Mira,9780778312994,0778312992,9,0,5,1,0,2012-02-12
4814,A GAME OF THRONES,In the frozen wastes to the north of Winterfel...,by George R.R. Martin,George RR Martin,,8.99,,Bantam,9780553573404,0553573403,10,0,44,0,0,2012-02-12


In [25]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
mass_market_paperback_weekly['rank'] = mass_market_paperback_weekly['rank'].astype(str)
filter_15 = mass_market_paperback_weekly['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [26]:
mass_market_paperback_weekly= mass_market_paperback_weekly[filter_15]

In [27]:
len(mass_market_paperback_weekly)

3900

In [28]:
#and a column indicating which list these come from
k2 = 3900
weekly_fiction_list = ['Mass Market Paperback Weekly']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3900

In [29]:
#add nyt_list as a column
mass_market_paperback_weekly['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
mass_market_paperback_weekly.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
4795,TOM CLANCY PRESENTS ACT OF VALOR,When the rescue of a C.I.A. operative leads to...,novelization by Dick Couch and George Galdorisi,novelization Dick Couch and George Galdorisi,,9.99,,Berkley,9780425259351,0425259358,11,0,3,0,0,2012-02-19,Mass Market Paperback Weekly
4796,FALLEN,The mother of the Georgia cop Faith Mitchell i...,by Karin Slaughter,Karin Slaughter,,7.99,,Dell,9780345528216,0345528212,12,0,1,1,0,2012-02-19,Mass Market Paperback Weekly
4797,COLD VENGEANCE,"Devastated by the murder of his wife, Special ...",by Douglas Preston and Lincoln Child,Douglas Preston and Lincoln Child,,9.99,,Grand Central,9780446554978,0446554979,13,0,1,0,0,2012-02-19,Mass Market Paperback Weekly
4798,LAST MAN STANDING,The special operative Joe Green is out to aven...,by Cindy Gerard,Cindy Gerard,,7.99,,Pocket Star,9781451606829,1451606826,14,0,1,1,0,2012-02-19,Mass Market Paperback Weekly
4799,THE JEFFERSON KEY,The ex-government operative Cotton Malone foil...,by Steve Berry,Steve Berry,,9.99,,Ballantine,9780345505521,0345505522,15,0,6,0,0,2012-02-19,Mass Market Paperback Weekly
4805,BONNIE,The forensic sculptor Eve Duncan learns more a...,by Iris Johansen,Iris Johansen,,7.99,,St. Martin’s,9780312651282,0312651287,1,0,1,0,0,2012-02-12,Mass Market Paperback Weekly
4806,THE GIRL WHO PLAYED WITH FIRE,In the second volume of the Millennium trilogy...,by Stieg Larsson,Stieg Larsson,,7.99,,Vintage Crime/Black Lizard,9780307476159,0307476154,2,0,71,0,0,2012-02-12,Mass Market Paperback Weekly
4807,THE GIRL WITH THE DRAGON TATTOO,A hacker and a journalist investigate the disa...,by Stieg Larsson,Stieg Larsson,,7.99,,Vintage Crime/Black Lizard,9780307949486,0307949486,3,0,83,0,0,2012-02-12,Mass Market Paperback Weekly
4808,A LADY NEVER SURRENDERS,When the youngest Sharpe sister hatches a plan...,by Sabrina Jeffries,Sabrina Jeffries,,7.99,,Pocket,9781451642452,1451642458,4,0,1,0,0,2012-02-12,Mass Market Paperback Weekly
4809,ONE FOR THE MONEY,The Jersey girl Stephanie Plum enters the busi...,by Janet Evanovich,Janet Evanovich,,8.99,,St. Martin’s,9780312547004,0312547005,5,0,6,0,0,2012-02-12,Mass Market Paperback Weekly


In [30]:
#save to csv
mass_market_paperback_weekly.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2013to2017_mass_market_paperback_weekly.csv", index=False)